In [1]:
import DistMLIP
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
import numpy as np

In [2]:
from DistMLIP.distributed.dist import Distributed

In [3]:
LFP = Structure.from_file("/data/kevinxhan/dist_chgnet/LiFePO4.cif")
LFP.make_supercell((10, 10, 10))
LFP_atoms = AseAtomsAdaptor().get_atoms(LFP)

In [4]:
lattice_matrix = np.array(LFP_atoms.get_cell())
cart_coords = np.array(LFP_atoms.get_positions(wrap=False))
frac_coords = np.array(LFP_atoms.get_scaled_positions(wrap=True))

pbc = np.array([1, 1, 1], dtype=int)
num_partitions = 2

dist_info = Distributed.create_distributed(cart_coords=cart_coords,
                                            frac_coords=frac_coords,
                                            lattice_matrix=lattice_matrix,
                                            num_partitions=num_partitions,  
                                            pbc=pbc,
                                            use_bond_graph=True,
                                            cutoff=5,
                                            three_body_cutoff=3,
                                            tol=1e-8,
                                            num_threads=128
                                        )

Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line graph items for partition 0
Reallocing # of line

In [5]:
dist_info

Distributed: 
    Total num atoms: 28000
    Total num edges: 1384000
    Bond graph exists: True
Partition 0:
	# of atom graph nodes: 16300. (2300 border nodes)
	# of atom graph edges: 692000	# of bond graph nodes: 153700. (21700 border nodes)
	# of bond graph edges: 1148000
Partition 1:
	# of atom graph nodes: 16300. (2300 border nodes)
	# of atom graph edges: 692000	# of bond graph nodes: 153700. (21700 border nodes)
	# of bond graph edges: 1148000